<a href="https://colab.research.google.com/github/sunkyoo/google_colab/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN을 이용한 MNIST 학습 및 PB 파일 생성

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile
from google.colab import files

MNIST 데이터셋 준비

In [0]:
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

hyper parameters

In [0]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100

Model configuration

In [0]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='data')
Y = tf.placeholder(tf.float32, [None, 10])

conv1 = tf.layers.conv2d(X, 32, [3, 3], padding="same", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, [2, 2], strides=2, padding="same")

conv2 = tf.layers.conv2d(pool1, 64, [3, 3], padding="same", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, [2, 2], strides=2, padding="same")

flat3 = tf.contrib.layers.flatten(pool2)
dense3 = tf.layers.dense(flat3, 256, activation=tf.nn.relu)

logits = tf.layers.dense(dense3, 10, activation=None)
final_tensor = tf.nn.softmax(logits, name='prob')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


Training

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples / batch_size)

print('Start learning!')
for epoch in range(training_epochs):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ',
            '{:.4f}'.format(total_cost/total_batch))

print('Learning finished!')

Start learning!
Epoch: 0001 Avg. cost =  0.1752
Epoch: 0002 Avg. cost =  0.0479
Epoch: 0003 Avg. cost =  0.0314
Epoch: 0004 Avg. cost =  0.0226
Epoch: 0005 Avg. cost =  0.0169
Epoch: 0006 Avg. cost =  0.0129
Epoch: 0007 Avg. cost =  0.0099
Epoch: 0008 Avg. cost =  0.0088
Epoch: 0009 Avg. cost =  0.0094
Epoch: 0010 Avg. cost =  0.0051
Epoch: 0011 Avg. cost =  0.0072
Epoch: 0012 Avg. cost =  0.0055
Epoch: 0013 Avg. cost =  0.0034
Epoch: 0014 Avg. cost =  0.0059
Epoch: 0015 Avg. cost =  0.0035
Epoch: 0016 Avg. cost =  0.0036
Epoch: 0017 Avg. cost =  0.0041
Epoch: 0018 Avg. cost =  0.0026
Epoch: 0019 Avg. cost =  0.0034
Epoch: 0020 Avg. cost =  0.0028
Learning finished!


Freeze variables and save pb file

In [0]:
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['prob'])
with gfile.FastGFile('./mnist_cnn.pb', 'wb') as f:
    f.write(output_graph_def.SerializeToString())

print('Save done!')

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.
Save done!


In [0]:
files.download('mnist_cnn.pb')